## Import libraries

In [ ]:
import os
import matplotlib.pyplot as plt
import json
import os
from statistics import mean, stdev
from pprint import pprint
import numpy as np

## Read json result files

In [ ]:
# result_base_path = "./store_with_leq_topk10"
result_base_path = "./store/dataset=airport/use_val_data=True/dirty-proportion=0.23/trainer-prior-type=uniform-0.1-learner-prior-type=uniform-0.9"

folders = os.listdir(result_base_path)
print(len(folders))

In [ ]:
results_dict = dict((trainer_type, {
    'Random': {'iter_accuracy': [],
               'iter_recall': [],
               'iter_precision': [],
               'iter_f1': [],
               'iter_mae_ground_model_error': [],
               'iter_mae_trainer_model_error': []
               },
    'ActiveLR': {'iter_accuracy': [],
                 'iter_recall': [],
                 'iter_precision': [],
                 'iter_f1': [],
                 'iter_mae_ground_model_error': [],
                 'iter_mae_trainer_model_error': []
                 },
    'StochasticBR': {'iter_accuracy': [],
                     'iter_recall': [],
                     'iter_precision': [],
                     'iter_f1': [],
                     'iter_mae_ground_model_error': [],
                     'iter_mae_trainer_model_error': []
                     },
    'StochasticUS': {'iter_accuracy': [],
                     'iter_recall': [],
                     'iter_precision': [],
                     'iter_f1': [],
                     'iter_mae_ground_model_error': [],
                     'iter_mae_trainer_model_error': []
                     }
}) for trainer_type in ['full-oracle', 'learning-oracle', 'bayesian'])

for folder in folders:
    trainer_type, sampling_method, _ = folder.split("_")
    with open(os.path.join(result_base_path, folder, "study_metrics.json"), 'r') as fp:
        study_metrics = json.load(fp)
        for metric_type in ['iter_accuracy', 'iter_recall', 'iter_precision', 'iter_f1',     'iter_mae_ground_model_error', 'iter_mae_trainer_model_error']:
            folder_name = None
            if sampling_method == 'RANDOM':
                folder_name = 'Random'
            elif sampling_method == 'ACTIVELR':
                folder_name = 'ActiveLR'
            elif sampling_method == 'STOCHASTICBR':
                folder_name = 'StochasticBR'
            elif sampling_method == 'STOCHASTICUS':
                folder_name = 'StochasticUS'
            results_dict[trainer_type][folder_name][metric_type].append([list(range(1, len(
                study_metrics[metric_type])+1)), study_metrics['elapsed_time'], study_metrics[metric_type]])

# pprint(results_dict['uninformed-bayesian']['ActiveLR']['iter_mae_ground_model_error'][0][2])
# pprint(results_dict['uninformed-bayesian']['ActiveLR']['iter_mae_trainer_model_error'][0][2])


## Compute Average time and accuracy

In [ ]:
'''For Random'''
average_dict = dict((trainer_type, {
    'Random': {'iter_accuracy': [],
               'iter_recall': [],
               'iter_precision': [],
               'iter_f1': [],
               'iter_mae_ground_model_error': [],
               'iter_mae_trainer_model_error': []
               },
    'ActiveLR': {'iter_accuracy': [],
                 'iter_recall': [],
                 'iter_precision': [],
                 'iter_f1': [],
                 'iter_mae_ground_model_error': [],
                 'iter_mae_trainer_model_error': []
                 },
    'StochasticBR': {'iter_accuracy': [],
                     'iter_recall': [],
                     'iter_precision': [],
                     'iter_f1': [],
                     'iter_mae_ground_model_error': [],
                     'iter_mae_trainer_model_error': []
                     },
    'StochasticUS': {'iter_accuracy': [],
                     'iter_recall': [],
                     'iter_precision': [],
                     'iter_f1': [],
                     'iter_mae_ground_model_error': [],
                     'iter_mae_trainer_model_error': []
                     }
}) for trainer_type in ['full-oracle', 'learning-oracle', 'bayesian'])

for trainer_type in ['full-oracle', 'learning-oracle', 'bayesian']:
    for sampling_type in ['Random', 'ActiveLR', 'StochasticBR', 'StochasticUS']:
        for metric_type in ['iter_accuracy', 'iter_recall', 'iter_precision', 'iter_f1', 'iter_mae_ground_model_error', 'iter_mae_trainer_model_error']:
            for exp_metrics_lst in zip(*results_dict[trainer_type][sampling_type][metric_type]):
                average_lst = []
                # print(exp_metrics_lst)

                max_len = max(len(exp_metric)
                              for exp_metric in exp_metrics_lst)
                # print(max_len)
                for idx in range(max_len):
                    candidate_lst = [exp_metric[idx] for exp_metric in exp_metrics_lst if idx < len(
                        exp_metric) and str(exp_metric[idx]) != 'nan']
                    if candidate_lst:
                        average_lst.append(mean(candidate_lst))
                    else:
                        average_lst.append(np.nan)
                average_dict[trainer_type][sampling_type][metric_type].append(
                    average_lst)
        # print(average_dict[sampling_type][metric_type])
# print(average_dict)


In [ ]:
# '''For Random'''
# average_dict = {'Random': {
#     'iter_accuracy': [],
#     'iter_recall': [],
#     'iter_precision': [],
#     'iter_f1': []
# },
#     'ActiveLR': {
#     'iter_accuracy': [],
#     'iter_recall': [],
#     'iter_precision': [],
#     'iter_f1': []
# }
# }
# for sampling_type in ['Random', 'ActiveLR']:
#     for metric_type in ['iter_accuracy', 'iter_recall', 'iter_precision', 'iter_f1']:
#         for val in zip(*results_dict[sampling_type][metric_type]):
#             idx = 0
#             average_vals = []
#             candidate_vals = []
#             end = False
#             while not end:
#                 end = True
#                 for exp_lst in val:
#                     if idx < len(exp_lst) and (str(exp_lst[idx]) != 'nan'):
#                         '''If any found then end is false'''
#                         candidate_vals.append(exp_lst[idx])
#                         end = False
#                 else:
#                     idx += 1
#                     # print(mean(candidate_vals), candidate_vals)
#                     print(candidate_vals)
#                     average_vals.append(mean(candidate_vals))

#             average_dict[sampling_type][metric_type].append(average_vals)


In [ ]:
# pprint(average_dict)

In [ ]:
for metric in ['accuracy', 'recall', 'precision', 'f1', 'mae_ground_model_error', 'mae_trainer_model_error']:
# for metric in ['accuracy', 'f1', 'recall', 'precision']:
    for trainer_type in ['bayesian']:
    # for trainer_type in ['full-oracle', 'learning-oracle', 'bayesian']:
    # for metric in ['accuracy','mae_model_error']:
        plt.figure(figsize=(12,7))
        # plt.subplot(2,1,1)
        for sampling_method in average_dict[trainer_type]:
            plt.plot(average_dict[trainer_type][sampling_method][f'iter_{metric}'][0], average_dict[trainer_type][sampling_method][f'iter_{metric}'][2], label=sampling_method)
        plt.xlabel('Iterations')
        plt.ylabel(metric)
        plt.ylim(bottom=0)
        plt.legend()
        # plt.subplot(2,1,2)
        # for sampling_method in average_dict[trainer_type]:
        #     plt.plot(average_dict[trainer_type][sampling_method][f'iter_{metric}'][1], average_dict[trainer_type][sampling_method][f'iter_{metric}'][2], label=sampling_method)
        # plt.xlabel('Time')
        # plt.ylabel(metric)
        # plt.legend()
        plt.title(f"Clean/Dirty Prediction {metric} from Learner Model with {trainer_type} trainer")
        plt.show()